Created by [SmirkyGraphs](https://smirkygraphs.github.io/). Code: [Github](https://github.com/SmirkyGraphs/Python-Notebooks). Source: [Cumberland Farms](https://www.cumberlandfarms.com/stores) | [HERE API](https://developer.here.com/).
<hr>

# Cumberland Farms - Isochrone Map

One common question raised when I showed my Dunkin' isochrone maps ([MA](https://preview.redd.it/l67vxd59dww61.png?width=1024&auto=webp&s=2e6ed451a00d9a3cf13fd5c821a7d957324e362b), [RI](https://preview.redd.it/x1fif9lss3w61.png?width=556&auto=webp&s=40e99416dbb8566a17022a8b3ae7790492260d53)) is often requests for other companies. One of which, was Cumberland Farms, a pretty popular north east gas station which also provides coffee at nearly all of their locations. I figured it would make a good follow up to the Dunkin' maps.

This code takes the data provided by the `cumberland-farms-locations` notebook and uses [HERE API](https://developer.here.com/) to generate isochrone lines for 568 store locations from across the east coast. The parameters used are `[150, 300, 600, 900]` seconds for the time range and `car` for the vehicle type.
<hr>

In [1]:
import json
import time
import requests
import pandas as pd
from pathlib import Path

import flexpolyline
import geopandas as gpd
from shapely.geometry import Polygon

In [2]:
def isoline_urls(df, time_range, tranport):
    coded_range = ','.join(str(x) for x in time_range)
    
    urls = {}
    for key, lat, lng in zip(df['StoreId'], df['Latitude'], df['Longitude']):
        url = "https://isoline.router.hereapi.com/v8/isolines?" + \
             f"apiKey={api_key}" + \
             f"&range[type]=time" + \
             f"&range[values]={coded_range}" + \
             f"&transportMode={transport_type}" + \
             f"&destination={lat},{lng}"

        urls[key] = url
        
    return urls

def isoline_response(key, url):
    r = requests.get(url)
    if r.status_code == 200:
        data = r.json()
    
    cols = ['id','store_key', 'location_lng', 'location_lat', 'seconds', 'geometry']
    gdf = gpd.GeoDataFrame(columns=cols, crs="EPSG:4326")
    lat = data['arrival']['place']['originalLocation']['lat']
    lng = data['arrival']['place']['originalLocation']['lng']
    
    for i, poly in enumerate(data['isolines']):
        seconds = (poly['range']['value'])
        polygon = poly['polygons'][0]['outer']
        geom = flexpolyline.decode(polygon)
        reverse_geom = [(xy[1], xy[0]) for xy in geom]
        gdf.loc[i] = [i, key, lng, lat, seconds, reverse_geom]
        
    return gdf

def collect_isolines(df, api_key, time_range, transport_type, sleep):
    # filter for already collected keys
    files = list(Path('./data/raw/').glob('*.geojson'))
    collected_keys = [x.stem for x in files]
    df = df[~df['StoreId'].astype(str).isin(collected_keys)]
    isolines = isoline_urls(df, time_range, transport_type)
    
    frames = []
    for key, url in isolines.items():
        gdf = isoline_response(key, url)
        gdf['geometry'] = gdf['geometry'].apply(Polygon)

        frames.append(gdf)
        gdf.to_file(f"./data/raw/{key}.geojson", driver='GeoJSON')
        time.sleep(sleep)
    
    return pd.concat(frames)

In [3]:
with open('./config.json', 'r') as f:
    config = json.load(f)
    
api_key = config['api_key']

time_range = [150, 300, 600, 900]
transport_type = 'car'

df = pd.read_csv('./data/files/cumberland_farms_clean.csv')
gdf = collect_isolines(df, api_key, time_range, transport_type, 5)
gdf.to_file("./data/clean/combined.geojson", driver='GeoJSON')

In [4]:
frames = []
for file in Path('./data/raw/').glob('*.geojson'):
    frames.append(gpd.read_file(file))
    
gdf = pd.concat(frames)

gdf.to_file("./data/clean/combined.geojson", driver='GeoJSON')